# how to control template behave

https://www.cppstories.com/2016/02/notes-on-c-sfinae/

# 1 Substitution failure is not an error (SFINAE)

What is SFINAE? Where can you use this metaprogramming technique? Are there any better alternatives in Modern C++? And how about Concepts from C++20?

> Substitution failure is not an error (SFINAE) refers to a situation in C++ where an invalid substitution of template parameters is not in itself an error. David Vandevoorde first introduced the acronym SFINAE to describe related programming techniques.

In [3]:
#include <iostream>
using namespace std;

In [6]:
struct Bar {
    typedef double internalType;  
};

In [7]:
template <typename T> 
typename T::internalType foo(const T& t) { 
    cout << "foo<T>\n"; 
    return 0; 
};

In [8]:
int main() {
    foo(Bar());
    foo(0); // << error!
};

input_line_15:3:5: error: no matching function for call to 'foo'
    foo(0); // << error!
    ^~~
input_line_14:2:26: note: candidate template ignored: substitution failure [with T = int]: type 'int'
      cannot be used prior to '::' because it has no members
typename T::internalType foo(const T& t) { 
         ~               ^


Interpreter Error: 

> candidate template ignored: substitution failure [with T = int]: type 'int'
      cannot be used prior to '::' because it has no members

We have one function template that returns T::internalType, and we call it with Bar and int param types.

The code, of course, will not compile. The first call of foo(Bar()); is a proper construction, but the second call generates the following error (GCC):

In [ ]:
When we make a simple correction and provide a suitable function for int types. As simple as:

In [10]:
int foo(int i) { cout << "foo(int)\n"; return 0; };

In file included from input_line_5:1:
In file included from /home/sz/anaconda3/envs/cling/include/xeus/xinterpreter.hpp:17:
In file included from /home/sz/anaconda3/envs/cling/include/xeus/xcomm.hpp:19:
In file included from /home/sz/anaconda3/envs/cling/include/nlohmann/json.hpp:42:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/iterator:64:
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/ostream:568:8: error: no member named 'setstate' in 'std::basic_ostream<char>'
        __out.setstate(ios_base::badbit);
        ~~~~~ ^
input_line_17:1:23: note: in instantiation of function template specialization
      'std::operator<<<std::char_traits<char> >' requested here
int foo(int i) { cout << "foo(int)\n"; return 0; };
                      ^


Interpreter Error: 

The code can be built and run.

Why is that?

When we added an overloaded function for the int type, the compiler could find a proper match and invoke the code. But in the compilation process, the compiler also ‘looks’ at the templated function header. This function is invalid for the int type, so why was there not even a warning reported (like we got when there was no second function provided)? To understand this, we need to look at the process of building the overload resolution set for a function call.

### Overload Resolution   
When the compiler tries to compile a function call (simplified):

* Perform a name lookup (see more @CppReference).
* For function templates the template argument values are deduced from the types of the actual arguments passed into the function.
1. All occurrences of the template parameter (in the return type and parameters types) are substituted with those deduced types.
2. When this process leads to invalid type (like int::internalType) the particular function is removed from the overload resolution set. (SFINAE)
* At the end, we have a list of viable functions that can be used for the specific call.
1. If this set is empty, then the compilation fails.
2. If more than one function is chosen, we have an ambiguity.
3. In general, the candidate function, whose parameters match the arguments most closely is the one that is called.

In our example: typename T::internalType foo(const T& t) was not a good match for int and it was rejected from overload resolution set. But at the end, int foo(int i) was the only option in the set, so the compiler did not report any problems.

## Where can I use it?   
I hope you get a basic idea what SFINAE does, but where can we use this technique? A general answer: whenever we want to select a proper function/specialization for a specific type.

* Call a function when T has a given method (like call toString() if T has toString method)
* Disallow narrowing or wrong conversions from wrapper types. For example, this is used to prevent std::variant from deducing the wrong types. See Everything You Need to Know About std::variant from C++17 - type conversions.
* ...

Ok, but how can we write such SFINAE expressions? Are there any helpers?

Let’s meet std::enable_if.
    
# 2 std::enable_if

One of the primary uses of SFINAE can be found through enable_if expressions.

enable_if is a set of tools, available in the Standard Library since C++11, that internally use SFINAE. They allow to include or exclude overloads from possible function templates or class template specialization.

For example:



In [ ]:
// C++11:
template <class T>
typename std::enable_if<std::is_arithmetic<T>::value, T>::type 
foo(T t) {
  std::cout << "foo<arithmetic T>\n";
  return t;
}

This function ‘works’ for all the types, that are arithmetic (int, long, float…). If you pass other types (for instance MyClass), it will fail to instantiate. In other words, template instantiations for non-arithmetic types are rejected from overload resolution sets. This construction might be used as a template parameter, function parameter or as a function return type.

enable_if<condition, T>::type will generate T, if the condition is true, or an invalid substitution if condition is false.

enable_if can be used along with type traits to provide the best function version based on the trait criteria.

Also please note that since C++14 and C++17 we have a nicer syntax and more compact. There’s no need to use ::type or ::value for enable_if or the traits, as there are _v and _t variable templates and template aliases introduced.

Our previous code can become:

In [ ]:
// C++17:
template <class T>
typename std::enable_if_t<std::is_arithmetic_v<T>, T> // << shorter!
foo(T t) {
  std::cout << "foo<arithmetic T>\n";
  return t;
}

In [ ]:
Please notice the use of std::enable_if_t and std::is_arithmetic_v.

See the full example:

In [13]:
#include <iostream>
#include <type_traits>

template <class T>
typename std::enable_if_t<std::is_arithmetic_v<T>, T> // << shorter!
foo(T t) {
  std::cout << "foo<arithmetic T>\n";
  return t;
};

template <class T>
typename std::enable_if_t<!std::is_arithmetic_v<T>, void>
foo(T t) {
  std::cout << "foo fallback\n";
};

int main() {
    foo(0);
    foo(std::string{});
};

In file included from input_line_1:1:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/new:40:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/exception:144:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/nested_exception.h:40:
In file included from /home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/move.h:55:
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/type_traits:137:31: error: no member named 'value' in
      'std::__not_<std::is_lvalue_reference<std::basic_ostream<char> &> >'
    : public conditional<_B1::value, __and_<_B2, _B3, _Bn...>, _B1>::type
                         ~~~~~^
/home/sz/anaconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/ostream:685:24: note

Interpreter Error: 

In [ ]:
alternative syntax about enbale_if is

In [ ]:
// if you think put enable_if_t in declaration is disgusting
template<typename T, typename = std::enable_if_t(std::is_arithmetic_v<T>, T)>
foo(T t) {
  std::cout << "foo<arithmetic T>\n";
  return t;
};

In [ ]:
another way is to use with using
template<typename T>
using EnableIfArithmetic =  std::enable_if_t(std::is_arithmetic_v<T>, T)

template<typename T, typename = EnableIfArithmetic<T>>
foo(T t) {
  std::cout << "foo<arithmetic T>\n";
  return t;
};

# Alternatives to SFINAE

### Any Disadvantages of SFINAE?   
SFINAE and enable_if are compelling features, but also it’s hard to get it right. Simple examples might work, but in real-life scenarios, you might get into all sorts of problems:

* Template errors: do you like reading template errors generated by the compiler? Especially when you use STL types?
* Readability
* Nested templates usually won’t work in enable_if statements
Here is a discussion at StackOverflow: Why should I avoid std::enable_if in function signatures.

Can we do something better?

We have at least three things:

* tag dispatching
* compile-time if
* and… Concepts!
Let’s review them briefly.

## Tag Dispatching   
This is a much more readable version of selecting which version of a function is called. First, we define a core function, and then we call version A or B depending on some compile-time condition.

In [ ]:
template <typename T>
int get_int_value_impl(T t, std::true_type) {
    return static_cast<int>(t+0.5f);
}

template <typename T>
int get_int_value_impl(T t, std::false_type) {
    return static_cast<int>(t);
}

template <typename T>
int get_int_value(T t) {
    return get_int_value_impl(t, std::is_floating_point<T>{});
}

In [ ]:
When you call get_int_value the compiler will then check the value of std::is_floating_point and then call the matching _impl function.

## Compile Time if(Since C++17)   
Since C++17 we have a new tool, build in the language, that allows you to check the condition at compile time - without the need to write complex templated code!

In a short form we can present it:

In [ ]:
template <typename T>
int get_int_value(T t) {
     if constexpr (std::is_floating_point<T>) {
         return static_cast<int>(t+0.5f);
     }
     else {
         return static_cast<int>(t);
     }
}

## Concepts - Since C++20   
With each C++ Standard revision, we get much better techniques and tools to write templates. In C++20 we’ll get a long-awaited feature, that will revolutionise the way we write templates!

With Concepts, you’ll be able to add constraints on the template parameters and get better compiler warnings.

One basic example:

In [ ]:
// define a concept:
template <class T>
concept SignedIntegral = std::is_integral_v<T> && std::is_signed_v<T>;

// use:
template <SignedIntegral T>
void signedIntsOnly(T val) { }

In the code above we first create a concept that describes types that are signed and integral. Please notice that we can use existing type traits. Later, we use it to define a function template that supports only types that match the concept. Here we don’t use typename T, but we can refer to the name of a concept.

### Summary   
In this post, we covered theory and examples of SFINAE - a template programming technique that allows you to reject code from the overload resolution sets. In raw form, this can be a bit complicated, but thanks to modern C++ we have many tools that can help: for example enable_if, std::declval and a few others. What’s more, if you’re lucky to work with the latest C++ standard, you can leverage if constexpr from C++17 and also Concepts from C++20.